In [1]:
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
import numpy as np
import gensim

#自作クラス
from CharToIndex import CharToIndex
from MyDatasets import Cross_Validation
from TimeChecker import TimeChecker


In [8]:


#9文字の中心を予測
class MyDataset(torch.utils.data.Dataset):
    def __init__(self,data,chars_file_path,device=torch.device('cpu')):
        self.data = data
        self.char2index = CharToIndex(chars_file_path)
        self.length = len(data['answer'])-8
        self.val_idx = torch.zeros(self.length+8,dtype=torch.long)
        self.ans_idx = torch.zeros(self.length+8,dtype=torch.long)
        self.device = device

        for i,char in enumerate(data['value']):
            self.val_idx[i] = self.char2index.get_index(char[0])

        for i,char in enumerate(data['answer']):
            self.ans_idx[i] = self.char2index.get_index(char)


    def __len__(self):
        return self.length


    def __getitem__(self,index):
        assert index < self.length
        return self.val_idx[index:index+9].to(device),self.ans_idx[index+4].to(device)


class Proofreader(nn.Module):
    def __init__(self, hidden_dim, class_num,n_layers,tokens,c2v):
        super(Proofreader, self).__init__()

        self.output_size = class_num
        self.hidden_dim = hidden_dim
        self.n_layers  = n_layers
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.mean_vector = self.make_mean_vector(class_num,tokens,c2v)


        self.rnn = nn.RNN(1024, self.hidden_dim, batch_first=True,bidirectional=True)
        self.dropout = torch.nn.Dropout(p=0.5)
        self.fc = nn.Linear(self.hidden_dim*2, class_num)
        self.fc2 = nn.Linear(class_num, class_num)

        self.to(self.device)



    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers*2, batch_size, self.hidden_dim)
        return hidden


    #意味ベクトルの作成
    def make_mean_vector(self,class_num,tokens,c2v):
        vec_size = c2v.vector_size
        m_vec = torch.zeros((class_num,vec_size),dtype=float)
        for i,char in enumerate(tokens.table):
            try:
                m_vec_i = np.copy(c2v.wv[char.decode()])
                m_vec[i] = torch.from_numpy(m_vec_i)
            except KeyError:
                m_vec[i] = 0
        return m_vec


    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size).to(self.device)

        x_m_vec = self.mean_vector[x[:,4]] #5文字目の意味ベクトル
        x_m_vec = torch.squeeze(x_m_vec)
        attention_mat = torch.zeros(size=(batch_size,self.mean_vector.shape[0]))

        for batch,vec in enumerate(x_m_vec):
            x_m_mat = torch.mul(vec,torch.ones_like(self.mean_vector)) #行列計算のためx_m_vecを列方向に複製
            cos_sims_x = torch.nn.functional.cosine_similarity(self.mean_vector,x_m_mat,dim=1) #cos類似度計算
            attention_mat[batch] =  torch.sigmoid(cos_sims_x).detach().clone() #シグモイド

        embed_x = self.mean_vector[x].float().to(device)
        out, hidden = self.rnn(embed_x, hidden)
        out = out[:,4,:]
        out = self.dropout(out)
        out = self.fc(out)
        out = torch.mul(out,attention_mat.to(device))
        out = self.fc2(out)

        return out

def train(model,train_dataloader,learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    batch_size = next(iter(train_dataloader))[0].size(0)
    running_loss = 0
    accuracy = 0

    model.train()
    for i,(x,y) in enumerate(train_dataloader):
        output = model(x)
        loss = criterion(output, y) #損失計算
        prediction = output.data.max(1)[1] #予測結果
        accuracy += prediction.eq(y.data).sum().item()/batch_size
        optimizer.zero_grad() #勾配初期化
        loss.backward(retain_graph=True) #逆伝播
        optimizer.step()  #重み更新
        running_loss += loss.item()

    loss_result = running_loss/len(train_dataloader)
    accuracy_result = accuracy/len(train_dataloader)

    return loss_result,accuracy_result


def eval(model,valid_dataloader,is_show_ans_pred=False):
    accuracy = 0
    batch_size = next(iter(valid_dataloader))[0].size(0)
    model.eval()
    for x,y in valid_dataloader:
        output = model(x)
        prediction = output.data.max(1)[1] #予測結果
        accuracy += prediction.eq(y.data).sum().item()/batch_size

    return accuracy/len(valid_dataloader)



In [ ]:

chars_file_path = "/net/nfs2/export/home/ohno/CR_pytorch/data/tegaki_katsuji/all_chars_3815.npy"
datas_file_path = "/net/nfs2/export/home/ohno/CR_pytorch/data/tegaki_katsuji/tegaki_distance.npz"
c2v_file_path   = "/net/nfs2/export/home/ohno/CR_pytorch/Wrod2Vec/context_2/word2vec_1024.model"
tokens = CharToIndex(chars_file_path)
data = np.load(datas_file_path,allow_pickle=True)

EMBEDDING_DIM = 10
HIDDEN_SIZE = 128
BATCH_SIZE = 64
VOCAB_SIZE = len(tokens)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tegaki_dataset = MyDataset(data,chars_file_path,device=device)
char2vector = gensim.models.Word2Vec.load(c2v_file_path)


cross_validation = Cross_Validation(tegaki_dataset)
p_acc_record=[]
p_loss_record=[]


In [31]:
def show_eval(model,valid_dataloader,tokens):
    collect_cnt=0
    whole_cnt = 0
    batch_size = next(iter(valid_dataloader))[0].size(0)
    model.eval()
    for i,(x,y) in enumerate(valid_dataloader):

        for idx in range(len(x)):
            output = model(x)
            # for item in x[idx]:
            #     print(tokens.get_decoded_char(item),end='')
            # print()
            for item in torch.topk(output[idx],1)[1]:
                print(tokens.get_decoded_char(item),end=' ')

            print(tokens.get_decoded_char(y[idx]))

            if torch.topk(output[idx],1)[1] == y[idx]:
                collect_cnt+=1
            whole_cnt+=1

    return collect_cnt/whole_cnt


In [32]:
##学習
train_dataset,valid_dataset = cross_validation.get_datasets(k_idx=0)
valid_dataloader=DataLoader(valid_dataset,batch_size=BATCH_SIZE,shuffle=False,drop_last=True)
proofreader = Proofreader(HIDDEN_SIZE,VOCAB_SIZE,1,tokens,char2vector)
proofreader.load_state_dict(torch.load("/net/nfs2/export/home/ohno/CR_pytorch/Examinations/Proof_Progs_wv/proof"))

p_val_accu = show_eval(proofreader,valid_dataloader,tokens)
print(f'val_accu:{p_val_accu:.5}')


利 利
用 用
い い
た た
だ だ
き き
ま ま
し し
て て
。 、
織 誠
に に
あ あ
り り
が が
と と
５ う
ご ご
ざ ざ
い い
ま ま
す す
。 。
さ さ
て て
１ 、
お お
客 客
様 様
の の
定 定
頼 額
・ ・
足 定
期 期
貯 預
金 金
は は
１ 、
右 右
新 記
の の
と と
お お
り り
満 満
期 期
を を
迎 迎
え え
ま ま
す す
の の
で で
。 、
ご ご
米 案
内 内
い い
た た
し し
来 ま
す す
。 。
今 今
後 後
と と
も も
一 一
康 層
の の
ご ご
愛 愛
顧 顧
を を
賜 賜
り り
ま ま
す す
よ よ
う う
お お
理 願
い い
出 申
し し
あ 上
け げ
ま ま
す す
。 。
（ （
払 払
度 戻
し し
お お
よ よ
び び
頂 預
け け
替 替
え え
の の
お お
ま 手
続 続
き き
の の
際 際
に に
必 必
要 要
と と
な な
る る
書 書
頼 類
等 等
） ）
貯 貯
金 金
諸 証
書 書
し 、
続 総
合 合
ロ 口
屋 座
通 通
頭 帳
（ （
無 無
過 通
圏 帳
型 型
続 総
合 合
ロ 口
屋 座
の の
場 場
合 合
は は
キ キ
ヤ ャ
い ッ
ジ シ
ュ ュ
カ カ
ー ー
ド ド
１ ）
（ 、
お お
届 届
け け
か 印
お お
よ よ
び び
ご ご
本 本
人 人
で で
あ あ
る る
こ こ
と と
を を
確 確
認 認
で で
き き
る る
証 証
明 明
書 書
ヒ 類
（ （
お お
名 名
前 前
、 ・
ご ご
住 住
所 所
、 ・
生 生
年 年
月 月
日 日
の の
入 入
」 っ
た た
運 運
転 転
意 免
評 許
証 証
等 等
し 、
法 法
人 人
名 名
義 義
。 の
場 場
合 合
は は
。 登
認 記
事 事
は 項
証 証
明 明
書 書
持 等
、 、
団 団
体 体
名 名
義 義
の の
場 場
合 合
は は
規 規
約 約
の の
手 写
し し
導 等
も も
あ あ
あ わ
せ せ
て て
お お
持 持
る ち
く く
だ だ
さ さ
い い
） ）
こ こ
の の
こ こ
と と
